
## MaxwellFDFD Data download
1. download data from link
2. make directory
3. unzip dataset


In [4]:
!curl -L -o maxwellfdfd.zip "https://drive.google.com/uc?id=14-Bl89OzRtLM1MCW2H81Xvivq8EvTrmB"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377    0   377    0     0     16      0 --:--:--  0:00:22 --:--:--    76
100 20.6M    0 20.6M    0     0   816k      0 --:--:--  0:00:25 --:--:-- 5774k


In [5]:
!mkdir maxwellfdfd
!unzip maxwellfdfd.zip -d ./maxwellfdfd

Archive:  maxwellfdfd.zip
  inflating: ./maxwellfdfd/test/binary_rl_fix_test_1004.csv  
  inflating: ./maxwellfdfd/test/binary_rl_fix_test_1005.csv  
  inflating: ./maxwellfdfd/test/binary_rl_fix_test_1002.csv  
  inflating: ./maxwellfdfd/test/binary_rl_fix_test_1003.csv  
  inflating: ./maxwellfdfd/test/binary_new_test_501.csv  
  inflating: ./maxwellfdfd/test/binary_rl_limit_test_1001.csv  
  inflating: ./maxwellfdfd/test/binary_new_test_1501.csv  
  inflating: ./maxwellfdfd/test/binary_test_1101.csv  
  inflating: ./maxwellfdfd/test/binary_rl_test_503.csv  
  inflating: ./maxwellfdfd/test/binary_rl_fix_1015.csv  
  inflating: ./maxwellfdfd/test/binary_rl_fix_1014.csv  
  inflating: ./maxwellfdfd/test/binary_rl_test_502.csv  
  inflating: ./maxwellfdfd/test/binary_rl_test_501.csv  
  inflating: ./maxwellfdfd/test/binary_new_test_501/305.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/240.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/31.tiff  
  inflating: ./ma

  inflating: ./maxwellfdfd/test/binary_new_test_501/362.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/159.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/270.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/335.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/17.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/266.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/323.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/289.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/40.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/118.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/231.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/374.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/303.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/246.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/495.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/37.ti

  inflating: ./maxwellfdfd/test/binary_new_test_501/300.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/183.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/479.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/34.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/496.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/212.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/357.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/8.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/63.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/204.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/341.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/438.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/75.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/253.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/316.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/195.tiff

  inflating: ./maxwellfdfd/test/binary_new_test_501/243.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/306.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/185.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/490.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/32.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/214.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/351.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/428.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/65.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/202.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/347.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/73.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/255.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/310.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/193.tiff  
  inflating: ./maxwellfdfd/test/binary_new_test_501/469.ti

  inflating: ./maxwellfdfd/test/binary_rl_test_503/358.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/122.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/437.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/95.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/175.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/460.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/319.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/56.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/227.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/362.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/159.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/270.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/335.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/17.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/266.tiff  
  inflating: ./maxwellfdfd/test/binary_rl_test_503/323.tiff  
  inflating

...

  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/380.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/695.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/20.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/482.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/178.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/528.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/197.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/894.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/744.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/314.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/601.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/251.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/77.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/986.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_tes

  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/874.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/527.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/177.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/659.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/209.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/97.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/966.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/435.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/120.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/570.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/823.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/989.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/78.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/423.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_tes

  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/510.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/843.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/18.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/906.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/455.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/681.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/394.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/814.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/547.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/117.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/402.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/951.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/926.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/475.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_te

  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/331.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/624.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/274.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/448.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/366.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/736.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/223.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/389.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/673.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/52.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/809.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/258.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/608.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/937.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_te

  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/516.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/453.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/900.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/687.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/238.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/392.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/668.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/541.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/111.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/49.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/812.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/957.tiff  
  inflating: ./maxwellfdfd/valid/binary_rl_fix_test_1001/404.tiff  


## ML(Random Forest regressor):
1. create model for Multi-output Random Forest regressor

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
%matplotlib inline

/opt/anaconda3/envs/maxwellfdfd-ai/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/envs/maxwellfdfd-ai/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#d

In [7]:
import os
data_dir = os.path.join(os.getcwd(), 'maxwellfdfd')
data_dir

'/Users/wonderit/projects/koreauniv/cem/maxwellfdfd-controlgan/maxwellfdfd'

In [ ]:
def compress_image(prev_image, n):
    if n < 2:
        return prev_image

    height = prev_image.shape[0] // n
    width = prev_image.shape[1] // n
    new_image = np.zeros((height, width), dtype="uint8")
    for i in range(0, height):
        for j in range(0, width):
            new_image[i, j] = prev_image[n * i, n * j]

    return new_image

DATASETS_TRAIN = [
    'binary_501',
    'binary_502',
    'binary_503',
    'binary_504',
    'binary_505',
    'binary_506',
    'binary_507',
    'binary_508',
    'binary_509',
    'binary_510',
    'binary_511',
    'binary_512',
    'binary_1001',
    'binary_1002',
    'binary_1003',
]

DATASETS_TEST = [
    'binary_new_test_501',
    'binary_new_test_1501',
    'binary_test_1101',
]

def cem_dataset(train=True):
    if train:
        DATAPATH = os.path.join(data_dir, 'train')
        DATASETS = DATASETS_TRAIN
    else:
        DATAPATH = os.path.join(data_dir, 'test')
        DATASETS = DATASETS_TEST
    width = 40
    height = 20
    input_data = []
    targets = []
    # load Train dataset
    for data in DATASETS:
        dataframe = pd.read_csv(os.path.join(DATAPATH, '{}.csv'.format(data)), delim_whitespace=False, header=None)
        dataset = dataframe.values

        # split into input (X) and output (Y) variables
        fileNames = dataset[:, 0]

        # 1. first try max
        dataset[:, 1:25] /= 2767.1

        # 2. Classification or Regression
        
        
        labels = dataset[:, 1:25]
        targets.extend(labels)

    print(f'Data Loading Finished. len : {len(input_data)}')
    return pd.DataFrame(input_data), pd.DataFrame(targets)


In [ ]:
X_train, y_train = cem_dataset(train=True)

In [ ]:
X_test, y_test = cem_dataset(train=False)

In [ ]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100))

In [ ]:
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)
y_pred_df=pd.DataFrame(y_pred)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
plt.scatter(y_test, y_pred)
plt.title('Scatter plot')
plt.xlabel('test')
plt.ylabel('pred')